In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import os, sys, inspect
import commands
import hep as hp

gamma = 0.577
d = 3.999
Delta = 2.0/(4.0-d)+np.log(4.0*np.pi)-gamma

In [3]:
def func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam3, lam6, lam7, lam8, lam9, lam10, lamh):
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo;
    k = s2phi*np.log(M22/M12)/(4.*np.pi)**2
    phi = 0.5*np.arcsin(s2phi)
    ba = 0.5*np.arcsin(-2.0*v*v*lam6/(MHo2-Mho2))
    Mu  = (M22-M12)*s2phi/(np.sqrt(2.0)*v)
    MH2 = M12*(np.sin(phi))**2+M22*(np.cos(phi))**2
    M332 = M12*(np.cos(phi))**2+M22*(np.sin(phi))**2
    lam1 = 0.5*(MHo2+Mho2-np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2))/(v**2)
    Mu22 = MH2-0.5*lam3*(v**2)
    Muh2 = M332-0.5*lam8*(v**2)
    lam4 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2+2.0*(MAo2-2.0*MH2))/(v**2)
    lam5 = 0.5*(np.sqrt((MHo2-Mho2)**2-4.0*v**4*lam6**2)+MHo2+Mho2-2.0*MAo2)/(v**2)        
    return k, Mu, Muh2, Mu22, lam1, lam4, lam5, phi, ba

In [4]:
def F(m12, m22):
    Fo = 0.5*(m12+m22)-((m12*m22)/(m12-m22))*np.log(m12/m22)
    return Fo

def T(phi, ba, M1, M2, MAo, Mho, MHo, alpha_em = 1./128., v=246.0, Mz=91.1876, Mw=80.385 ):
    Sphi2 = np.sin(phi)*np.sin(phi); Cphi2 = np.cos(phi)*np.cos(phi); 
    Sba2 = np.sin(ba)*np.sin(ba); Cba2 = np.cos(ba)*np.cos(ba);
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo; Mw2 = Mw*Mw; Mz2= Mz*Mz;
    A = 1.0/(16.0*np.pi*np.pi*alpha_em*v*v)
    B = Cba2*F(M12,Mho2)+Sba2*F(M12,MHo2)+F(M12,MAo2)
    C = Cba2*F(M22,Mho2)+Sba2*F(M22,MHo2)+F(M22,MAo2)
    D = 0.5*Sphi2*Cphi2*F(M12,M22)+Cba2*F(Mho2,MAo2)+Sba2*F(MHo2,MAo2)
    E = F(Mz2,MHo2)-F(Mw2,MHo2)-F(Mz2,Mho2)+F(Mw2,Mho2)
    y = A*(Cphi2*B+Sphi2*C-D+3.0*Cba2*E)
    return y


In [5]:
def W(q2, m2):    
    w = np.sqrt(4.0*m2-q2)
    return w

def Y(q2, m1, m2):
    y = np.sqrt(((m1+m2)*(m1+m2)-q2)*((m1-m2)*(m1-m2)-q2))
    return y

def Yo(q2, m1, m2):
    y = np.sqrt(((m1+m2)*(m1+m2)-q2)*(q2-(m1-m2)*(m1-m2)))
    return y

def Z(q2, m2):
    z = np.sqrt(4.0*q2-m2)
    return z

def Zo(q2, m2):
    z = np.sqrt(m2-4.0*q2)
    return z
    
def B22(q, m1, m2):
    m12 = m1*m1; m22 = m2*m2; q2 = q*q;
    if(q==m1):
        if(q<m2/2 and q>0):
            A = (m22-q2)**3/(q2*q2)-3.0*(m22*m22-q2*q2)/q2+3.0*(m22*m22+q2*q2)/(m22-q2)
            B = 8.0/3.0+Delta-np.log(m1*m2)
            b = 1.0/12.0*(3.0*q2/2.0+9.0*m22/2.0-m22*m22/q2-m2*m22*(Zo(q2,m22))**3*np.log((m2+Zo(q2,m22))/(m2-Zo(q2,m22)))/(q2*q2)+A*np.log(m2/m1)-B*q2)
            return b
        else:
            A = (m22-q2)**3/(q2*q2)-3.0*(m22*m22-q2*q2)/q2+3.0*(m22*m22+q2*q2)/(m22-q2)
            B = 8.0/3.0+Delta-np.log(m1*m2)
            b = 1.0/12.0*(3.0*q2/2.0+9.0*m22/2.0-m22*m22/q2-m2*m22*(Z(q2,m22))**3*np.arctan2(Z(q2,m22),m2)/(q2*q2)+A*np.log(m2/m1)-B*q2)
            return b
    if(m1==m2):
        A = 8.0/3.0+Delta-2.0*np.log(m1)
        b = 1.0/12.0*(8.0*m12-2.0*(W(q2,m12))**3*np.arctan2(q,W(q2,m12))/q-A*q2)
        return b
    else:    
        if((m1-m2)*(m1-m2) > q2):
            A = -(m12-m22)*(m12-m22)/q2+5.0/2.0*(m12+m22)
            B = (Y(q2,m1,m2))**3/(2.0*q2*q2)*np.log((m12+m22-q2+Y(q2,m1,m2))/(m12+m22-q2-Y(q2,m1,m2)))
            C = (m12-m22)**3/(q2*q2)-3.0*(m12*m12-m22*m22)/q2+3.0*(m12*m12+m22*m22)/(m12-m22)
            D = 8.0/3.0+Delta-np.log(m1*m2)
            b = 1.0/12.0*(A-B+C*np.log(m1/m2)-D*q2)
            return b
        else:
            A = -(m12-m22)*(m12-m22)/q2+5.0/2.0*(m12+m22)
            B = (Yo(q2,m1,m2))**3/(2.0*q2*q2)*np.arctan2(((m12+m22-q2)*Yo(q2,m1,m2)),((m12+m22-q2)*(m12+m22-q2)-2.0*m12*m22))
            C = (m12-m22)**3/(q2*q2)-3.0*(m12*m12-m22*m22)/q2+3.0*(m12*m12+m22*m22)/(m12-m22)
            D = 8.0/3.0+Delta-np.log(m1*m2)
            b = 1.0/12.0*(A-B+C*np.log(m1/m2)-D*q2)
            return b
    
def Bo(q, m1, m2):
    m12 = m1*m1; m22 = m2*m2; q2 = q*q;
    if(q==m1):
        if(q<m2/2 and q>0):
            A = m22/q2*(m22-3.0*q2)/(m22-q2)*np.log(m2/q)
            b = 1.0+m1*Zo(q2,m22)*np.log((m2+Zo(q2,m22))/(m2-Zo(q2,m22)))/q2-A
            return b
        else:
            A = m22/q2*(m22-3.0*q2)/(m22-q2)*np.log(m2/q)
            b = 1.0-m1*Z(q2,m22)*np.arctan2(Z(q2,m22),m2)/q2-A
            return b
    if(m1==m2):
        b = 2.0-2.0*W(q2,m12)*np.arctan2(q,W(q2,m12))/q
        return b
    else:
        if((m1-m2)*(m1-m2) > q2):
            A = ((m12+m22)*q2-(m12-m22)*(m12-m22))/((m12-m22)*q2)
            B = Y(q2,m1,m2)/(2.0*q2)*np.log((m12+m22-q2+Y(q2,m1,m2))/(m12+m22-q2-Y(q2,m1,m2)))
            b = 1.0+B+A*np.log(m1/m2)
            return b
        else:
            A = ((m12+m22)*q2-(m12-m22)*(m12-m22))/((m12-m22)*q2)
            B = Yo(q2,m1,m2)/(2.0*q2)*np.arctan2(((m12+m22-q2)*Yo(q2,m1,m2)),((m12+m22-q2)*(m12+m22-q2)-2.0*m12*m22))
            b = 1.0-B+A*np.log(m1/m2)
            return b
        
def S(phi, ba, M1, M2, MAo, Mho, MHo, v=246.0, Mz=91.1876, Mw=80.385):
    Sphi2 = np.sin(phi)*np.sin(phi); Cphi2 = np.cos(phi)*np.cos(phi); 
    Sba2 = np.sin(ba)*np.sin(ba); Cba2 = np.cos(ba)*np.cos(ba); C2phi = np.cos(2.0*phi)
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo; Mw2 = Mw*Mw; Mz2= Mz*Mz;
    A = 1.0/(np.pi*Mz2)
    B = B22(Mz,Mho,MAo)+B22(Mz,Mz,MHo)-B22(Mz,Mz,Mho)#-Mz2*Bo(Mz,Mz,MHo)+Mz2*Bo(Mz,Mz,Mho)
    C = Sba2*B22(Mz,MHo,MAo)+2.0*Sphi2*Cphi2*B22(Mz,M1,M2)
    D = 0.5*Cphi2*(C2phi-3.0)*B22(Mz,M1,M1)-0.5*Sphi2*(C2phi+3.0)*B22(Mz,M2,M2)
    y = A*(C+Cba2*B+D)
    return y

def U(phi, ba, M1, M2, MAo, Mho, MHo, v=246.0, Mz=91.1876, Mw=80.385):
    Sphi2 = np.sin(phi)*np.sin(phi); Cphi2 = np.cos(phi)*np.cos(phi); 
    Sba2 = np.sin(ba)*np.sin(ba); Cba2 = np.cos(ba)*np.cos(ba); C2phi = np.cos(2.0*phi)
    M12 = M1*M1; M22 = M2*M2; MAo2 = MAo*MAo; Mho2 = Mho*Mho; MHo2 = MHo*MHo; Mw2 = Mw*Mw; Mz2= Mz*Mz;
    A = 1.0/(np.pi*Mw2)
    B = B22(Mw,M1,MAo)+Sba2*B22(Mw,M1,MHo)+Cba2*B22(Mw,Mho,M1)
    C = B22(Mw,M2,MAo)+Sba2*B22(Mw,M2,MHo)+Cba2*B22(Mw,Mho,M2)
    D = B22(Mw,Mw,MHo)-B22(Mw,Mw,Mho)-Mw2*Bo(Mw,Mw,MHo)+Mw2*Bo(Mw,Mw,Mho)
    E = Cphi2*B22(Mw,M1,M1)+Sphi2*B22(Mw,M2,M2)
    s = S(phi, ba, M1, M2, MAo, Mho, MHo, v, Mz, Mw)
    y = A*(Cphi2*B+Sphi2*C+Cba2*D-2.0*E)-s
    return y


In [6]:
v = 246.0
s2phi = 0.0
M1 = 700.0
M2 = 1000.0
MAo = 705.0
Mho = 125.0
MHo = 902.0
lam2 = 1.3000000E-01    # lambda2Input
lam3 = 1.0000000E-03    # lambda3Input
lam6 = 0.000000E-03     # lambda6Inputd 
lam7 = 0.000000E+00     # lambda7Input
lam8 = 0.0000000E-01    # lambda8Input
lam9 = 0.000000E+00     # lambda9Input
lam10 = 0.000000E+00    # lambda10Input
lamh = 0.000000E+00     # lambdahInput
                
k, Mu, Muh2, Mu22, lam1, lam4, lam5, phi, ba = func(s2phi, M1, M2, MAo, Mho, MHo, v, lam2, lam3, lam6, lam7, lam8, lam9, lam10, lamh)

t = T(phi, 90.0*np.pi/180.0, M1, M2, MAo, Mho, MHo, alpha_em = 1./128., v=246.0, Mz=91.1876, Mw=80.385 )
s = S(phi, 90.0*np.pi/180.0, M1, M2, MAo, Mho, MHo, v=246.0, Mz=91.1876, Mw=80.385)
u = U(phi, 90.0*np.pi/180.0, M1, M2, MAo, Mho, MHo, v=246.0, Mz=91.1876, Mw=80.385)

print 'T = ',t
print 'S = ',s
print 'U = ',u

T =  0.0179202218408
S =  0.00756566870471
U =  3.54665545141e-05
